In [1]:
#import some useful packages
import numpy as np
import matplotlib.pyplot as plt
from qiskit import IBMQ, schedule
from qiskit_ibm_provider import IBMProvider
from qiskit.circuit import Gate, Parameter, QuantumCircuit
from qiskit import pulse, schedule
from qiskit.tools.jupyter import *
from qiskit_ibm_provider.job import job_monitor
from qiskit.pulse import Gaussian
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split
%config InlineBackend.figure_formats = ['svg']
import time 
import os
import csv

import warnings
warnings.filterwarnings('ignore')

import os
import csv

IBMQ.save_account('c28063071324a27bf0f536ec2d6c376b146467db302e3b3039204d028f1ecdd776dea90d6c104302c8b2808d51786c51948e360d6b6c19d7b74a45b0da2e2671')
provider = IBMQ.load_account()

# provider = IBMProvider()
backend = provider.get_backend('ibm_lagos')

configrc.store_credentials:WARNING:2023-08-18 22:59:35,128: Credentials already present. Set overwrite=True to overwrite.


In [2]:
#get information about backend
backend_defaults = backend.defaults()
backend_properties = backend.properties()
backend_config = backend.configuration()

In [3]:
#hyperparameters
qubit = 0
f01 = backend.properties().qubits[qubit][2].value * 1e9
anhar = backend.properties().qubits[qubit][3].value * 1e9
f12 = f01 + anhar

qubit = 0
pi = np.pi
cos = np.cos
sin = np.sin
NUM_SHOTS = 20000
scale_factor = 1e-7

In [4]:
f12 = 4895197000.149713
amp01 = 0.4150600327660328
amp12 = 0.3290598588545396
dur12 = 144
sig12 = dur12/4
dur01 = 160
sig01 = dur01/4

In [5]:
def r01_sched(theta, phi):
    with pulse.build(backend=backend, default_alignment='sequential',
                     name=r'$R^{(01)}$') as r01_pulse:
        drive_chan = pulse.drive_channel(qubit)
        pulse.set_frequency(f01, drive_chan)
        with pulse.phase_offset(-phi, drive_chan):
            pulse.play(pulse.Gaussian(duration=dur01,
                                      amp=amp01*theta/np.pi,
                                      sigma=sig01,
                                      name=r'$X_{\pi/2}^{(01)}$'), drive_chan)

    return r01_pulse

def r12_sched(theta, phi):
    with pulse.build(backend=backend, default_alignment='sequential',
                     name=r'$R^{(12)}$') as r12_pulse:
        drive_chan = pulse.drive_channel(qubit)
        pulse.set_frequency(f12, drive_chan)
        with pulse.phase_offset(-phi, drive_chan):
            pulse.play(pulse.Gaussian(duration=dur12,
                                      amp=amp12*theta/np.pi,
                                      sigma=sig12,
                                      name=r'$X_{\pi/2}^{(01)}$'), drive_chan)

    return r12_pulse

In [6]:
#get information about default X gate
calibrations = backend_defaults.instruction_schedule_map
calibrations.get('x', qubit), calibrations.get('sx', qubit)

(Schedule((0, Play(Drag(duration=160, sigma=40, beta=-1.0789757601572947, amp=0.384205678130623, angle=0.0, name='Xp_d0'), DriveChannel(0), name='Xp_d0')), name="x"),
 Schedule((0, Play(Drag(duration=160, sigma=40, beta=-1.1013886682356577, amp=0.1889428893585572, angle=0.023297991511911236, name='X90p_d0'), DriveChannel(0), name='X90p_d0')), name="sx"))

In [7]:
#some useful pulse
with pulse.build(backend=backend, default_alignment='sequential', name=r'$X_{\pi}^{(12)}$') as xpi12:
    drive_chan = pulse.drive_channel(qubit)
    pulse.set_frequency(f12, drive_chan)
    pulse.play(pulse.Gaussian(duration=dur12,
                            amp=amp12,
                            sigma=sig12,
                            name=r'$X_{\pi}^{(12)}$'), drive_chan)

with pulse.build(backend=backend, default_alignment='sequential', name=r'$X_{\pi/2}^{(12)}$') as x90_12:
    drive_chan = pulse.drive_channel(qubit)
    pulse.set_frequency(f12, drive_chan)
    pulse.play(pulse.Gaussian(duration=dur12,
                            amp=amp12/2,
                            sigma=sig12,
                            name=r'$X_{\pi/2}^{(12)}$'), drive_chan)
    
with pulse.build(backend=backend, default_alignment='sequential', name=r'$X_{\pi}^{(01)}$') as xpi01:
    drive_chan = pulse.drive_channel(qubit)
    pulse.set_frequency(f01, drive_chan)
    pulse.play(pulse.Drag(duration=160,
                               amp=0.4150600327660328,
                               sigma=40,
                               beta=-1.1001415145124334,
                               angle=0.0,
                               name=r'$X_{\pi}^{(01)}$'), drive_chan)

with pulse.build(backend=backend, default_alignment='sequential', name=r'$X_{\pi/2}^{(01)}$') as x90_01:
    drive_chan = pulse.drive_channel(qubit)
    pulse.set_frequency(f01, drive_chan)
    pulse.play(pulse.Drag(duration=160,
                               amp=0.20392106238832894,
                               sigma=40,
                               beta=-1.2553416232209385,
                               angle=0.02115022932604328,
                               name=r'$X_{\pi/2}^{(01)}$'), drive_chan)   
# Create the three circuits
Xpi01 = Gate("$X_{\pi}^{01}$", 1, [])
Xpi12 = Gate("$X_{\pi}^{12}$", 1, [])
X90_01 = Gate(r'$X_{\pi/2}^{(01)}$', 1, [])
X90_12 = Gate(r'$X_{\pi/2}^{(12)}$', 1, [])
# 0 state
zero = QuantumCircuit(1, 1)
zero.measure(0, 0)

# 1 state
one = QuantumCircuit(1, 1)

#one.append(Xpi01, [0])
#one.add_calibration(Xpi01, (0,), r01(pi, 0), [])
one.x(0)
one.measure(0, 0)

# 2 state 
two = QuantumCircuit(1, 1)
two.x(0)
#two.append(Xpi01, [0])
two.append(Xpi12, [0])
two.measure(0, 0)
#two.add_calibration(Xpi01, (0,), r01(pi, 0), [])
two.add_calibration(Xpi12, (0,), r12_sched(pi, 0), [])

discr_circ = [zero, one, two]

# Generalize phase tracking protocol with many jobs

In [8]:
def protocol(order):
    Xpi01 = Gate("$X_{\pi}^{01}$", 1, [])
    Xpi12 = Gate("$X_{\pi}^{12}$", 1, [])
    X90_01 = Gate(r'$X_{\pi/2}^{(01)}$', 1, [])
    X90_12 = Gate(r'$X_{\pi/2}^{(12)}$', 1, [])

    zero = QuantumCircuit(1, 1)
    zero.measure(0, 0)

    one = QuantumCircuit(1, 1)
    one.x(0)
    one.measure(0, 0)

    two = QuantumCircuit(1, 1)
    two.x(0)
    two.append(Xpi12, [0])
    two.measure(0, 0)
    two.add_calibration(Xpi12, (0,), r12_sched(pi, 0), [])

    discr_circ = [zero, one, two]
    exp_circs = []
    params = []
    for i in range(97):
        phi = np.random.default_rng().uniform(0, pi, len(order))
        theta = np.random.default_rng().uniform(0, pi, len(order))
        
        params.append(np.concatenate((theta, phi)))
        
        qc = QuantumCircuit(1,1)
        qc.append(X90_01, [0])
        qc.append(X90_12, [0])
        qc.add_calibration(X90_01, (0,), x90_01, [])
        qc.add_calibration(X90_12, (0,), x90_12, [])
        
        i = 0
        
        for level in order:
            if level == '1':
                name = r"$R^{01}$" + fr"$(\theta_{i+1}$" + ',' + fr'$\phi_{i+1})$'
                R01 = Gate(name, 1, [])
                qc.append(R01, [0])
                qc.add_calibration(R01, (0,), r01_sched(theta[i], phi[i]), [])
            elif level == '2':
                name = r"$R^{12}$" + fr"$(\theta_{i+1}$" + ',' + fr'$\phi_{i+1})$'
                R12 = Gate(name, 1, [])
                qc.append(R12, [0])
                qc.add_calibration(R12, (0,), r12_sched(theta[i], phi[i]), [])
            else:
                print('Out of level')
                break
            i = i + 1
        qc.measure(0,0)
        exp_circs.append(qc)
    exp_circs = discr_circ + exp_circs
    
    return exp_circs, params



In [9]:
#create a list of random order
import random
def create_order(length):
    order = ''
    for i in range(length):
        order = order + str(random.randint(1,2))
    return order

def create_orders(length):
    orders = set()
    for i in range(20):
        orders.add(create_order(length))
    return orders
orders = []
for i in range(4, 15, 1):
    orders += random.sample(list(create_orders(i)), k=5)

In [10]:
#run many jobs with many orders
tg = time.localtime(time.time())
dd = str(tg.tm_mday)
mm = str(tg.tm_mon) 
yy = str(tg.tm_year)
folder = f"{yy}_{mm}_{dd}"
id_list = []
for order in orders:
    exp_circs, params = protocol(order)
    tg = time.localtime(time.time())
    params_file = f"./{folder}/{order}_params_{yy}_{mm}_{dd}_lagos.csv"
    np.savetxt(params_file, params, delimiter=',')
    job = backend.run(exp_circs, meas_level=1, meas_return='single', shots=NUM_SHOTS)
    id_list.append(job.job_id())


In [27]:
#save job's id and orders

jobid_file = f"./{folder}/job_id_0_{yy}_{mm}_{dd}_lagos.txt"
with open(jobid_file, 'w') as f:
    for job_id in id_list:
        f.write(job_id)
        f.write('\n')

file = f"./{folder}/order_0_{yy}_{mm}_{dd}_lagos.txt"
with open(file, 'w') as f:
    for order in orders:
        f.write(order)
        f.write('\n')